# 추천  시스템 프로토타입

## 데이터 전처리 작업

In [3]:
import pandas as pd

#### 데이터 불러오기

In [2]:
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv')

In [3]:
data.head()

,Unnamed: 0,name,address,category,rating,id
0,0,설레임,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5dee59b58f87a842bc875a3a
1,1,설레임,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5dee59b58f87a842bc875a3a
2,2,설레임,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5c381b1c5745ec219264a906
3,3,설레임,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5c381b1c5745ec219264a906
4,4,설레임,"제주특별자치도 제주시 애월읍 일주서로 5999-1, 2층",애월읍,5.0,5df37ace8f87a842bc889e8c


In [4]:
data.drop(columns={'Unnamed: 0', 'id'}, inplace=True)

#### 리뷰 갯수

In [5]:
cnt_review = pd.DataFrame(data.value_counts('name'))

In [6]:
cnt_review.head()

,0
name,
카프리,72324
설레임,41304
돈이랑,25704
만월당,12576
고집돌우럭 중문점,12040


#### 평균 별점

In [7]:
avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

In [8]:
avg_star.head()

,rating
name,
(주)삼다도횟집,4.350000
11시11분,4.689655
17번가,4.500000
1950김프로,3.611111
204유부당,4.625000


## 추천 시스템 함수화

#### 전처리 및 가중치 함수화

In [1]:
def make_review_data(data, area, percentile=0.6):

    cnt_review = pd.DataFrame(data.value_counts('name'))
    avg_star = data.pivot_table('rating',index='name',aggfunc='mean')

    # 평점/리뷰갯수 정보를 가지는 temp 변수 생성
    temp = avg_star.merge(cnt_review,left_index=True,right_index=True,how='left')
    temp.columns = ['rating','num']
    
    temp= temp.fillna(0)

    
    m = temp['num'].quantile(percentile)
    C = temp['rating'].mean()

    
    #가충치 함수를 람다함수로 처리
    temp['weighted_rating'] = temp.apply((lambda x: (x['num']/(x['num']+m)*x['rating'])+(m/(m+x['num'])*C)), axis=1)
    
    
    # 음식점 이름 + 지역을 기준으로 그룹화 시켜서 
    name_area =pd.DataFrame(data.groupby(['name'])['category'].first())
    
    # 평점/리뷰갯수에 음식점 이름 + 지역 병합
    temp=temp.merge(name_area,left_index=True,right_index=True)
    
    # 사용자 위치에 해당되는 정보만 정리
    temp=temp[temp['category']==area]
    
    # 정렬후 반환
    return temp[['rating','weighted_rating','num','category']].sort_values('weighted_rating', ascending=False)[:10]

## 실제 구현 부분

In [6]:
# 해당 지점을 DB랑 연결
data = pd.read_csv('./data/네이버맵음식점평점_최종.csv')

if __name__=='__main__':
    print('사용자 위치를 입력해주세요')
    print()
    input_data_area=input()
    #음식업종
    info = make_review_data(data, input_data_area)
    
    info.reset_index(inplace=True)
    
    print()
    print()
    
    for i in range(10):
        print(i,' 순위')
        print('음식점 이름 : ', info['name'][i], '평균 평점 : ', info['weighted_rating'][i])
        print()
        print()

사용자 위치를 입력해주세요

제주시


0  순위
음식점 이름 :  돈마니 평균 평점 :  4.948036094432248


1  순위
음식점 이름 :  제주공항본점공항뚝배기 평균 평점 :  4.9257343490769046


2  순위
음식점 이름 :  우돈향 평균 평점 :  4.913549278755223


3  순위
음식점 이름 :  반디파스타 평균 평점 :  4.907040921971566


4  순위
음식점 이름 :  멋진만남 평균 평점 :  4.892274052519437


5  순위
음식점 이름 :  더팔당제주 평균 평점 :  4.886307159630288


6  순위
음식점 이름 :  다미진 평균 평점 :  4.8843150583999275


7  순위
음식점 이름 :  어부지리 평균 평점 :  4.874172899064839


8  순위
음식점 이름 :  덤장돌삼겹 평균 평점 :  4.872764303439099


9  순위
음식점 이름 :  서프로와 평균 평점 :  4.872386922846265




In [13]:
info

,name,rating,weighted_rating,num,category
0,돈마니,5.000000,4.948036,296,제주시
1,제주공항본점공항뚝배기,4.953598,4.925734,528,제주시
2,우돈향,4.946759,4.913549,432,제주시
3,반디파스타,4.915744,4.907041,1626,제주시
4,멋진만남,4.920833,4.892274,480,제주시
5,더팔당제주,5.000000,4.886307,119,제주시
6,다미진,4.885732,4.884315,9504,제주시
7,어부지리,4.940000,4.874173,200,제주시
8,덤장돌삼겹,4.990991,4.872764,111,제주시
9,서프로와,4.895431,4.872387,569,제주시


In [14]:
 make_review_data(data, input_data_area)

,rating,weighted_rating,num,category
name,,,,
돈마니,5.000000,4.948036,296,제주시
제주공항본점공항뚝배기,4.953598,4.925734,528,제주시
우돈향,4.946759,4.913549,432,제주시
반디파스타,4.915744,4.907041,1626,제주시
멋진만남,4.920833,4.892274,480,제주시
더팔당제주,5.000000,4.886307,119,제주시
다미진,4.885732,4.884315,9504,제주시
어부지리,4.940000,4.874173,200,제주시
덤장돌삼겹,4.990991,4.872764,111,제주시


In [16]:
 make_review_data(data, '제주시')

,rating,weighted_rating,num,category
name,,,,
돈마니,5.000000,4.948036,296,제주시
제주공항본점공항뚝배기,4.953598,4.925734,528,제주시
우돈향,4.946759,4.913549,432,제주시
반디파스타,4.915744,4.907041,1626,제주시
멋진만남,4.920833,4.892274,480,제주시
더팔당제주,5.000000,4.886307,119,제주시
다미진,4.885732,4.884315,9504,제주시
어부지리,4.940000,4.874173,200,제주시
덤장돌삼겹,4.990991,4.872764,111,제주시


In [17]:
 make_review_data(data, '서귀포시')

,rating,weighted_rating,num,category
name,,,,
중문색달통갈치,4.903427,4.896247,1926,서귀포시
중문흑돼지천국,4.895531,4.887343,1656,서귀포시
칼초네화덕피자,4.891478,4.882546,1502,서귀포시
포트그릴스테이크하우스,4.907843,4.881592,510,서귀포시
찰리스,4.880623,4.865730,867,서귀포시
썬라이즈135,4.872483,4.851533,596,서귀포시
오또도넛,4.868805,4.850642,686,서귀포시
짱실장돈까스,4.978261,4.844752,92,서귀포시
탐나돈정육점식당,4.844340,4.836235,1484,서귀포시
